## 모델 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00


In [ ]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
#from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.lib import tokenization_kobert
import json
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [ ]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai
# with open('/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/save_src_tk.pickle','rb') as f:
#     src_tokenizer = pickle.load(f)
# there's an error in this code when activating in local

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai


In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/kobert_tokenizer_add/added_tokens.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=12290
mi=8002
for i in range(8002,12291):
    src_tokenizer.add_tokens(dict_swap[i])

In [ ]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

## 테스트데이터 불러오기

In [ ]:
def read_input(path):
    with open(path, 'rb') as f:
        test_dict = json.load(f)
    test_standard = []
    test_dialect = []
    for pairs in test_dict['utterance']:
        test_standard.append(pairs['standard_form'])
        test_dialect.append(pairs['dialect_form'])

    return test_standard ,test_dialect

In [ ]:
test_s ,test_d= read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/test_data_big.json")

In [ ]:
def translate(text):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    tmp = trg_tokenizer.decode(output.cpu())
    return tmp

In [ ]:
print("size of full : ",len(test_s))

size of full :  12451


In [ ]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result :      ",tg_text)
    print("ground_truth :",test_s[i])
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)
        break

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


source_text :  언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
result :       언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터     
ground_truth : 언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터


iteration num :  0


## scoring

In [ ]:
!pip install "sacrebleu[ko]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-ko-dic: filename=mecab_ko_dic-1.0.0-py3-none-any.whl size=33424394 sha256=19426d9f2ebe97480c10894ec1d12dcc3a36a1f5754a9748fde4f5c12b0fc4df
  Stored in directory: /root/.cache/pip/wheels/0d/46/b8/996432c9998690a5a66fb82f6499222e48a23ff5b6c7d80322
Successfully built mecab-ko-dic


In [ ]:
from sacrebleu.metrics import BLEU

In [ ]:
def score_B(sr_text, tg_text):

    candidate=[tg_text]
    reference=[[sr_text]]

    bleu = BLEU()

    bleuscore = bleu.corpus_score(candidate,reference).score
    return bleuscore
    #bleu_save.append(bleuscore)

In [ ]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result      : ",tg_text)
    print("ground_truth: ",test_s[i])
    print("bleu_socre  :  {:.2f}".format(score_B(tg_text.strip(),test_s[i].strip())))
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)

source_text :  언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
result      :  언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터     
ground_truth:  언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터
bleu_socre  :  100.00


iteration num :  0
source_text :  어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예
result      :  어 명절 설 명절 추석 명절 이렇게 나누어서 해볼 거예요   
ground_truth:  어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요
bleu_socre  :  65.80


source_text :  자 그믄 이제부터 얘기해 보게예
result      :  자 그러면 이제부터 얘기해 봐요         
ground_truth:  자 그러면 이제부터 얘기해 봐요
bleu_socre  :  100.00


source_text :  어 그믄 언니네 설 명절 때 음식 어떵 해
result      :  어 그러면 언니네 설 명절 때 음식 어떻게 해       
ground_truth:  어 그러면 언니네 설 명절 때 음식 어떻게 해
bleu_socre  :  100.00


source_text :  음식을 막 준비했던 기억이 나게
result      :  음식을 막 준비했던 기억이 나겠어요        
ground_truth:  음식을 막 준비했던 기억이 나
bleu_socre  :  66.87


source_text :  밀가룬지 쌀가룬지를 모르크라  근데 그거를 푹
result      :  밀가루인지 쌀가루인지 모르겠어 근데 그거를 푹      
ground_truth:  밀가룬지 쌀가룬지를 모르겠어  근데 그거를 푹
bleu_socre  :  50.81


source_text :  반죽을 해그내 날 납작하게
result      :  반죽을 해서 날 납작하게             
ground_

KeyboardInterrupt: ignored